# 1.2 고급 웹 크롤러(1)
---

- daum IT 뉴스 크롤러
    - https://news.daum.net/breakingnews/digital
    
1. 뉴스 목록 첫 페이지
2. 상세 페이지
3. 다음 페이지 이동

---

#### 3.8.1.1 daum_news_list_1 실습

In [5]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os
import re
import string

In [9]:
REG_DATE = '20210715'
news_main_url = 'https://news.daum.net/breakingnews/digital?regDate={}'.format(REG_DATE)

In [30]:
# daum 뉴스 목록 첫 페이지

def first_page(news_main_url):
    full_url = []
    response = requests.get(news_main_url)
    root = lxml.html.fromstring(response.content)
    
    for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
        a = li.xpath('div/strong/a')[0]
        url = a.get('href')
        # print(url, a.text)
        full_url.append(url)
        print(url, a.text)
        
    return full_url

In [31]:
first_page(news_main_url)

https://v.daum.net/v/20210715093534641 KT, '5G 단독모드' 상용화..갤럭시 3개 단말 제공
https://v.daum.net/v/20210715093414565 구글코리아, 국내 머신러닝 개발자 양성 위한 부트캠프 개최
https://v.daum.net/v/20210715093402551 아이폰12 대박에 고무된 애플, 아이폰13 물량 20% 늘렸다
https://v.daum.net/v/20210715093253494 "기가지니로 제주삼다수 주문"..KT-광동제약 음성주문서비스 MOU
https://v.daum.net/v/20210715093208461 에쓰오일, 창사 이래 최초 무재해 800만인시 달성
https://v.daum.net/v/20210715093201456 블록체인의 진화..금융ICT융합학회 참관기 [여기는 논설실]
https://v.daum.net/v/20210715093123431 '말로 하는' 클럽하우스에서 쪽지도 보낸다..'백채널' 도입
https://v.daum.net/v/20210715093105412 KT엔지니어링, 데이터센터 구축 사업 표준운영절차 정립
https://v.daum.net/v/20210715093042383 KT, 광동제약과 AI 생수 간편 주문 서비스 만든다
https://v.daum.net/v/20210715093039380 밀리의서재, 수면 오디오북 '굿나잇 밀리' 오픈
https://v.daum.net/v/20210715093038378 7월 넷째주 빈집털이 가장 많아.."계량기·SNS까지 모니터링"
https://v.daum.net/v/20210715092937301 "기가지니로 제주삼다수 주문"..KT-광동제약 음성주문서비스 MOU
https://v.daum.net/v/20210715092853276 에쓰오일, 창사이래 첫 무재해 800만 人時 달성
https://v.daum.net/v/20210715092803242 자녀가 보는 콘텐츠, 부모에게 알린다..넷플릭스, 신

['https://v.daum.net/v/20210715093534641',
 'https://v.daum.net/v/20210715093414565',
 'https://v.daum.net/v/20210715093402551',
 'https://v.daum.net/v/20210715093253494',
 'https://v.daum.net/v/20210715093208461',
 'https://v.daum.net/v/20210715093201456',
 'https://v.daum.net/v/20210715093123431',
 'https://v.daum.net/v/20210715093105412',
 'https://v.daum.net/v/20210715093042383',
 'https://v.daum.net/v/20210715093039380',
 'https://v.daum.net/v/20210715093038378',
 'https://v.daum.net/v/20210715092937301',
 'https://v.daum.net/v/20210715092853276',
 'https://v.daum.net/v/20210715092803242',
 'https://v.daum.net/v/20210715092733229']

In [29]:
# daum 뉴스 목록 상세 페이지
def get_detail(url):
    body = []
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·]'
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    
    # 본문 내용
    for p in root.xpath('//*[@id="harmonyContainer"]/section/p'):
        if p.text:
            body.append(re.sub(punc, '', p.text)) # 특수문자 제거
    
    full_body = ' '.join(body)
    
    return full_body



get_detail(first_page(news_main_url)[1])

'15일 블룸버그 보도에 따르면 애플은 최근 공급 업체에 아이폰13 물량 9000만대 생산을을 요청했다 애플은 지난 몇 년간 아이폰 초기 생산량을 7500만대 수준으로 유지해왔다 블룸버그는 신종 코로나바이러스 감염증코로나19 백신 접종 시작 후 나오는 첫 신형 아이폰인 만큼 추가 수요에 대한 애플의 기대감이 작용했다고 해석했다 지난해 출시된 애플의 첫 5G5세대 통신 스마트폰인 아이폰12 시리즈는\xa07개월 만에 1억대가 팔리는 호조를 보였다 블룸버그는 업계에 정통한 관계자들을 인용해 세계적 반도체 수급난에도 애플은 큰 영향을 받지 않는다 워낙 대형 고객이라 대만의 반도체 제조사 TSMC 등이 미리 공급량을 준비하기 때문이라면서 중국 화웨이가 미국 제재로 사업이 위축된 상황도 고려됐다고 전했다 아이폰13은 오는 9월 출시가 유력하다 아이폰13 시리즈는 전작인 아이폰12와 마찬가지로 △아이폰13 미니54인치 △아이폰1361인치 △아이폰13 프로61인치 △아이폰13 프로 맥스67인치 라인업으로 예상된다 최수진 한경닷컴 기자 naive@hankyungcom\xa0'

In [32]:
# 뉴스 첫 페이지 목록과 상세 페이지
page = 1
max_page = 0

In [33]:
response = requests.get('https://news.daum.net/breakingnews/digital?page={}?regDate={}'.format(page,REG_DATE))
root = lxml.html.fromstring(response.content)

for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
    a = li.xpath('div/strong/a')[0]
    url = a.get('href')
    article = get_detail(url)
    
    print(f'URL : {url}')
    print(f'TITLE : {a.text}')
    print(f'ARTICLE : {article}')
    print('-' * 100)

URL : https://v.daum.net/v/20210715093959851
TITLE : 소니, 무선헤드폰 'WH-1000XM4 미드나잇 블루' 신규 색상 출시
ARTICLE : 서울뉴스1 이창규 기자  소니코리아가 국내 무선 헤드폰 시장에서 8년 연속 1위를 차지했던 WH1000XM4의 미드나잇 블루Midnight Blue 컬러를 국내 출시한다고 15일 밝혔다 WH1000XM4 미드나잇 블루의 판매가는 45만9000원이며 소니코리아 온오프라인 공식 판매점과 이커머스 플랫폼 쿠팡에서 구매할 수 있다 WH1000XM4 미드나잇 블루는 기존 Δ블랙 Δ플래티넘 실버 Δ리미티드 에디션의 사일런트 화이트 컬러에 이어 4번째로 국내에 출시되는 신규 컬러다  기존 WH1000XM4 제품과 마찬가지로 소니의 고성능 HD 노이즈 캔슬링 프로세서 QN1HD Noise Cancelling Processor QN1에 듀얼 노이즈 센서 기술과 최신 블루투스 오디오 시스템 온 칩Bluetooth Audio System on Chip이 새롭게 탑재됐다  또한 대화 목소리를 감지해 자동으로 음악을 일시 정지하는 스피크 투 챗Speaktochat 기능을 비롯해 헤드폰을 벗지 않고도 조작이 가능한 다양한 스마트 기술을 지원한다  아울러 소니코리아는 국내 1위 오디오북 구독 서비스 윌라와 함께 WH1000XM4의 정품등록 프로모션을 진행한다 제품 구매 후 9월12일까지 소니코리아 고객지원 사이트에서 정품등록 및 사은품 신청을 완료한 고객에게 최신 베스트셀러 오디오북과 각 분야별 최고 강사의 클래스까지 즐길 수 있는 윌라 오디오북클래스 무제한 6개월 이용권을 증정한다 yellowapollo@news1kr 
----------------------------------------------------------------------------------------------------
URL : https://v.daum.net/v/20210715093835795
TITLE : 엔씨소프트, 